## Reimplementing our first example from scratch in TensorFlow

In [1]:
import tensorflow as tf
import math
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist

2023-02-09 22:36:30.310717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        #Create a matrix, W(Weight), of shape (input_size, output_size), initialized with random values.
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        #Create a vector, b(bias), of shape (output_size,), initialized with zeros.
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    
    #Apply the forward pass
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    #Convenient method for retrieving the layer's weights
    @property
    def weights(self):
        return [self.W, self.b]

In [3]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [4]:
#A mock Keras Model
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

2023-02-09 22:36:33.831430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 **Batch Generator**

In [5]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size] 
        self.index += self.batch_size
        return images, labels

**Running One Training Step**

In [6]:
def one_training_step(model, images_batch, labels_batch): 
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights) #will define this function below
    return average_loss

In [7]:
# learning_rate = 1e-3

# def update_weights(gradients, weights): 
#     for g, w in zip(gradients, weights):
#         w.assign_sub(g * learning_rate) #assign_sub is the equivalent of -= for TensorFlow variables.

> Instead of doing sth like in cell 14 where we implement a weight update step by hand, we would use an Optimizer instance from Keras, as follows:


In [8]:
optimizer = optimizers.SGD(learning_rate=1e-3) #SGD -> Stochastic Gradient Descent

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

### Here is the full training loop

> But first, what is epoch? 
>> An epoch of training is when the model goes through the entire training data set one time, dividing it into mini-batches and performing the training step on each mini-batch. The full training loop is simply repeating this process, with multiple epochs, until the desired level of accuracy is achieved.

In [9]:
def fit(model, images, labels, epochs, batch_size=128): 
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch) 
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

# Test Drive

In [10]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 4.65
loss at batch 100: 2.25
loss at batch 200: 2.19
loss at batch 300: 2.09
loss at batch 400: 2.26
Epoch 1
loss at batch 0: 1.92
loss at batch 100: 1.88
loss at batch 200: 1.81
loss at batch 300: 1.70
loss at batch 400: 1.86
Epoch 2
loss at batch 0: 1.59
loss at batch 100: 1.58
loss at batch 200: 1.48
loss at batch 300: 1.41
loss at batch 400: 1.53
Epoch 3
loss at batch 0: 1.33
loss at batch 100: 1.34
loss at batch 200: 1.22
loss at batch 300: 1.20
loss at batch 400: 1.29
Epoch 4
loss at batch 0: 1.13
loss at batch 100: 1.16
loss at batch 200: 1.02
loss at batch 300: 1.04
loss at batch 400: 1.12
Epoch 5
loss at batch 0: 0.98
loss at batch 100: 1.02
loss at batch 200: 0.89
loss at batch 300: 0.92
loss at batch 400: 0.99
Epoch 6
loss at batch 0: 0.87
loss at batch 100: 0.91
loss at batch 200: 0.79
loss at batch 300: 0.83
loss at batch 400: 0.90
Epoch 7
loss at batch 0: 0.79
loss at batch 100: 0.82
loss at batch 200: 0.71
loss at batch 300: 0.76
loss at batch 40

## Evaluating the model performance

In [11]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.82
